# 데이콘 baseline 바탕으로 한 감성 분석

In [8]:
import pandas as pd
import numpy as np
import torch
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk

In [9]:
data = pd.read_csv('D:/코드/nsmc/ratings_train.csv') 
data.head() 

,id,review,label
0,id,document,label
1,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
2,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
3,10265843,너무재밓었다그래서보는것을추천한다,0
4,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0


In [12]:
# 결측치 제거
data = data.dropna(subset=['id', 'review', 'label'])
data.head()

,id,review,label
0,id,document,label
1,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
2,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
3,10265843,너무재밓었다그래서보는것을추천한다,0
4,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0


## train / validation set 나누기

In [13]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해
val.reset_index(inplace=True)

In [14]:
train

,index,id,review,label
0,114905,9205512,"엔야의 only time노래가 흐를때 눈물이,,,,",1
1,101496,10245132,보영이누나 연기너무잘하신다 앞으로응원할게요홧팅♥,1
2,133923,7863603,둘이 실제부부이고 남자주인공이 각본 감독까지 했다능 정말 특이하고 감정적인,1
3,119674,160230,이런 영화만들돈으로 차라리 불쌍한사람들이나 도와줘라,0
4,119504,8712892,tv 전기세가 무지 아까웠다!!!,0
...,...,...,...,...
112492,34500,6678085,드라마 안본 사람도 즐길수 있는 영화,1
112493,143684,3634150,정말 재미 없다. 야한장면이라도 많이 나왔으면 10점 줄껀데 옛다 1점..,0
112494,43724,9756679,아 빵점주고싶다ㅠㅠ.,0
112495,69981,5598276,지루해요,0


## 데이터 전처리

In [15]:
# 한글과 공백을 뺀 나머지를 제거하여 preprocessed 라는 열 추가
train['preprocessed'] = train['review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)
train['preprocessed'] = train['preprocessed'].str.replace(" +", " ") # 다중 공백 제거
train[:5]

,index,id,review,label,preprocessed
0,114905,9205512,"엔야의 only time노래가 흐를때 눈물이,,,,",1,엔야의 노래가 흐를때 눈물이
1,101496,10245132,보영이누나 연기너무잘하신다 앞으로응원할게요홧팅♥,1,보영이누나 연기너무잘하신다 앞으로응원할게요홧팅
2,133923,7863603,둘이 실제부부이고 남자주인공이 각본 감독까지 했다능 정말 특이하고 감정적인,1,둘이 실제부부이고 남자주인공이 각본 감독까지 했다능 정말 특이하고 감정적인
3,119674,160230,이런 영화만들돈으로 차라리 불쌍한사람들이나 도와줘라,0,이런 영화만들돈으로 차라리 불쌍한사람들이나 도와줘라
4,119504,8712892,tv 전기세가 무지 아까웠다!!!,0,전기세가 무지 아까웠다


In [16]:
# 형태소 분석기 사용
from konlpy.tag import Okt

okt = Okt()
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in train['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내옴.
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 기본형 추출)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가
train["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가

In [17]:
train.head()

,index,id,review,label,preprocessed,tokenized_stem
0,114905,9205512,"엔야의 only time노래가 흐를때 눈물이,,,,",1,엔야의 노래가 흐를때 눈물이,엔야 의 노래 가 흐르다 때 눈물 이
1,101496,10245132,보영이누나 연기너무잘하신다 앞으로응원할게요홧팅♥,1,보영이누나 연기너무잘하신다 앞으로응원할게요홧팅,보영 이 누나 연기 너무 잘 하다 앞 으로 응원 하다 홧팅
2,133923,7863603,둘이 실제부부이고 남자주인공이 각본 감독까지 했다능 정말 특이하고 감정적인,1,둘이 실제부부이고 남자주인공이 각본 감독까지 했다능 정말 특이하고 감정적인,둘 이 실제 부부 이고 남자 주인공 이 각본 감독 까지 하다 능 정말 특이하다 감정...
3,119674,160230,이런 영화만들돈으로 차라리 불쌍한사람들이나 도와줘라,0,이런 영화만들돈으로 차라리 불쌍한사람들이나 도와줘라,이렇다 영 화만 들 돈 으로 차라리 불쌍하다 사람 들 이나 도와주다
4,119504,8712892,tv 전기세가 무지 아까웠다!!!,0,전기세가 무지 아까웠다,전 기세 가 무지 아깝다


In [38]:
# 품사 설정하고 명사/동사/형용사/부사 추출하여 main_pos열에 추가

main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in train['review']: # 리뷰들을 하나씩 가져옴
    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받음
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가
train["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가

In [39]:
train.iloc[:5]

,index,id,review,label,preprocessed,tokenized_stem,main_pos
0,114905,9205512,"엔야의 only time노래가 흐를때 눈물이,,,,",1,엔야의 노래가 흐를때 눈물이,엔야 의 노래 가 흐르다 때 눈물 이,엔야 노래 흐를 때 눈물
1,101496,10245132,보영이누나 연기너무잘하신다 앞으로응원할게요홧팅♥,1,보영이누나 연기너무잘하신다 앞으로응원할게요홧팅,보영 이 누나 연기 너무 잘 하다 앞 으로 응원 하다 홧팅,보영 누나 연기 너무 하신다 앞 응원 할게요 홧팅
2,133923,7863603,둘이 실제부부이고 남자주인공이 각본 감독까지 했다능 정말 특이하고 감정적인,1,둘이 실제부부이고 남자주인공이 각본 감독까지 했다능 정말 특이하고 감정적인,둘 이 실제 부부 이고 남자 주인공 이 각본 감독 까지 하다 능 정말 특이하다 감정...,둘 실제 부부 남자 주인공 각본 감독 했다 능 정말 특이하고 감정
3,119674,160230,이런 영화만들돈으로 차라리 불쌍한사람들이나 도와줘라,0,이런 영화만들돈으로 차라리 불쌍한사람들이나 도와줘라,이렇다 영 화만 들 돈 으로 차라리 불쌍하다 사람 들 이나 도와주다,이런 화만 돈 차라리 불쌍한 사람 도와줘라
4,119504,8712892,tv 전기세가 무지 아까웠다!!!,0,전기세가 무지 아까웠다,전 기세 가 무지 아깝다,기세 무지 아까웠다


## 벡터화

In [40]:
X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.label #training 데이터에서 라벨 추출

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # transform

## 모델 학습

In [42]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression() #객체에 모델 할당
model.fit(X_train_vec, y_train) #모델 학습

LogisticRegression()

## 모델 성능 검증(validation set으로)

In [43]:
# preprocessed
val['preprocessed'] = val['review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 한글과 공백을 제외하고 모두 제거
val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in val['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
val["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['review']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val.head()

,index,id,review,label,preprocessed,tokenized_stem,main_pos
0,79069,3941123,전재미있기만한데;;끝이조금찝찝하긴햇지만;; 석방이라니 ;;,1,전재미있기만한데끝이조금찝찝하긴햇지만 석방이라니,전 재미있다 한 데끝 이 조금 찝찝하다 햇 지만 석방 이라니,전 재미있기만 데 끝 조금 찝찝하긴 햇 석방
1,77378,2767469,안타깝다...,0,안타깝다,안타깝다,안타깝다
2,5523,6594040,진심 대박 재밌어요 겟 스마트2도 많이 사랑해 주세요,1,진심 대박 재밌어요 겟 스마트도 많이 사랑해 주세요,진심 대박 재밌다 겟 스마트 도 많이 사랑 하다 줄다,진심 대박 재밌어요 스마트 도 많이 사랑 해 주세요
3,68239,1326622,볼만한데 홍콩영화의 한계다,1,볼만한데 홍콩영화의 한계다,볼 만 한 데 홍콩영화 의 한계 다,볼 데 홍콩영화 한계
4,148418,6600497,이거 예전에 봤는데 감독에게 하고싶은말은 단하나다. 사람가지고 놀리지마라.,0,이거 예전에 봤는데 감독에게 하고싶은말은 단하나다 사람가지고 놀리지마라,이 거 예전 에 보다 감독 에게 하다 싶다 단 하나 다 사람 가지 고 놀리다,거 예전 봤는데 감독 하고 싶은말은 하나 사람 가지 놀리지마라


In [44]:
X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.label #validation 데이터에서 라벨 추출

X_val_vec = vectorizer.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform

In [45]:
#run model
y_pred = model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

['1' '0' '1' ... '0' '1' '0']


In [46]:
from sklearn import metrics
print('accuracy =', metrics.accuracy_score(y_val, y_pred)) # 정확도 확인

accuracy = 0.826555374809995
